КАТЕГОРИИ В ДАННЫХ О НЕДВИЖИМОСТИ

Давайте определим число уникальных категорий в каждом столбце нашей таблицы melb_df. Для этого создадим вспомогательную таблицу unique_counts:



In [27]:
import pandas as pd
melb_data = pd.read_csv('data/melb_data.csv')
melb_df = melb_data.copy(deep=True)
melb_df['Date'] = pd.to_datetime(melb_df['Date'],dayfirst=True)


def get_street_type(address):
    exclude_list = ['N', 'S', 'W', 'E']
    address_split = address.split(' ')
    street_type = address_split[-1]
    if street_type in exclude_list:
        street_type = address_split[-2]
    return street_type
stret_types = melb_df['Address'].apply(get_street_type)
poplar_types = stret_types.value_counts().nlargest(10).index
melb_df['StreetType'] = stret_types.apply(lambda x: x if x in poplar_types else 'other')


WeekdaySale = melb_df['WeekdaySale'] = melb_df['Date'].dt.dayofweek

poplur_sellerg = melb_df['SellerG'].value_counts().nlargest(49).index
# создал список из 49 самых популярных компаний
melb_df['SellerG'] = melb_df['SellerG'].apply(lambda x: x if x in poplur_sellerg else 'other')

melb_df['MonthSale'] = melb_df['Date'].dt.month


In [28]:
# создаём пустой список
unique_list = []
# пробегаемся по именам столбцов в таблице
for col in melb_df.columns:
    # создаём кортеж (имя столбца, число уникальных значений)
    item = (col, melb_df[col].nunique(),melb_df[col].dtypes) 
    # добавляем кортеж в список
    unique_list.append(item) 
# создаём вспомогательную таблицу и сортируем её
unique_counts = pd.DataFrame(
    unique_list,
    columns=['Column_Name', 'Num_Unique', 'Type']
).sort_values(by='Num_Unique',  ignore_index=True)
# выводим её на экран
display(unique_counts)

,Column_Name,Num_Unique,Type
0,Type,3,object
1,WeekdaySale,5,int64
2,Method,5,object
3,Regionname,8,object
4,Bathroom,9,int64
5,Rooms,9,int64
6,StreetType,11,object
7,Car,11,int64
8,Bedroom,12,int64
9,MonthSale,12,int64


In [29]:
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car']
for col in melb_df.columns:
    if melb_df[col].nunique() < 150 and col not in cols_to_exclude:
        melb_df[col] = melb_df[col].astype('category')
melb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 27 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Unnamed: 0     13580 non-null  int64         
 1   index          13580 non-null  int64         
 2   Suburb         13580 non-null  object        
 3   Address        13580 non-null  object        
 4   Rooms          13580 non-null  int64         
 5   Type           13580 non-null  category      
 6   Price          13580 non-null  float64       
 7   Method         13580 non-null  category      
 8   SellerG        13580 non-null  category      
 9   Date           13580 non-null  datetime64[ns]
 10  Distance       13580 non-null  float64       
 11  Postcode       13580 non-null  int64         
 12  Bedroom        13580 non-null  int64         
 13  Bathroom       13580 non-null  int64         
 14  Car            13580 non-null  int64         
 15  Landsize       1358

ПОЛУЧЕНИЕ АТРИБУТОВ CATEGORY

У типа данных category есть свой специальный аксесcор **cat**, который позволяет получать информацию о своих значениях и преобразовывать их. Например, с помощью атрибута этого аксессора **categories** мы можем получить список уникальных категорий в столбце Regionname:

In [30]:
melb_df['Regionname'].cat.categories

Index(['Eastern Metropolitan', 'Eastern Victoria', 'Northern Metropolitan',
       'Northern Victoria', 'South-Eastern Metropolitan',
       'Southern Metropolitan', 'Western Metropolitan', 'Western Victoria'],
      dtype='object')

In [31]:
melb_df['Regionname'].cat.codes

0        2
1        2
2        2
3        2
4        2
        ..
13575    4
13576    6
13577    6
13578    6
13579    6
Length: 13580, dtype: int8

С помощью метода аксессора **rename_categories()** можно легко переименовать текущие значения категорий. Для этого в данный метод нужно передать словарь, ключи которого — старые имена категорий, а значения — новые.



In [32]:
print(melb_df['Type'].cat.categories)
melb_df['Type'] = melb_df['Type'].cat.rename_categories({
    'u': 'unit',
    't': 'townhouse',
    'h': 'house'
})
melb_df['Type'].cat.categories

Index(['h', 't', 'u'], dtype='object')


Index(['house', 'townhouse', 'unit'], dtype='object')

А теперь представим ситуацию, что появилась новая партия домов и теперь мы продаём и квартиры (flat). Создадим объект Series new_houses_types, в котором будем хранить типы зданий новой партии домов. Преобразуем тип new_houses_types в такой же тип, как и у столбца Type в таблице melb_data, и выведем результат на экран:

In [33]:
new_houses_types = pd.Series(['unit', 'house', 'flat', 'flat', 'house'])
new_houses_types = new_houses_types.astype(melb_df['Type'].dtype)
new_houses_types

0     unit
1    house
2      NaN
3      NaN
4    house
dtype: category
Categories (3, object): ['house', 'townhouse', 'unit']

Решить эту проблему на самом деле не сложно. Можно добавить категорию flat в столбец Type с помощью метода акссесора **cat.add_categories(),** в который достаточно просто передать имя новой категории:

In [34]:
melb_df['Type'] = melb_df['Type'].cat.add_categories('flat')
new_houses_types = pd.Series(['unit', 'house', 'flat', 'flat', 'house'])
new_houses_types.astype(melb_df['Type'].dtype)

new_houses_types

0     unit
1    house
2     flat
3     flat
4    house
dtype: object

In [40]:
result_list = melb_df['Suburb'].value_counts().nlargest(119).index
melb_df['Suburb'] = melb_df['Suburb'].apply(lambda x: x if x in result_list else 'other')
melb_df['Suburb'].astype('category')
melb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 27 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Unnamed: 0     13580 non-null  int64         
 1   index          13580 non-null  int64         
 2   Suburb         13580 non-null  object        
 3   Address        13580 non-null  object        
 4   Rooms          13580 non-null  int64         
 5   Type           13580 non-null  category      
 6   Price          13580 non-null  float64       
 7   Method         13580 non-null  category      
 8   SellerG        13580 non-null  category      
 9   Date           13580 non-null  datetime64[ns]
 10  Distance       13580 non-null  float64       
 11  Postcode       13580 non-null  int64         
 12  Bedroom        13580 non-null  int64         
 13  Bathroom       13580 non-null  int64         
 14  Car            13580 non-null  int64         
 15  Landsize       1358